In [1]:
import pickle
from helpers import treat_article, compute_plagiarism_from_articles, compute_many_to_many_sim
from glob import glob
from nltk.corpus import stopwords

In [ ]:
### Parameters

ENG_STOPWORDS = set(stopwords.words("english"))
N_GRAM = 3
GAP_TOLERANCE = 5 
PADDING = 20

In [2]:
### File paths 
fr_path = "./txt files/french.txt"
en_path = "./txt files/english.txt"
it_path = "./txt files/italian.txt"
es_path = "./txt files/spanish.txt"
de_path = "./txt files/deutch.txt"

fr_article = treat_article(fr_path,ENG_STOPWORDS,N_GRAM)
en_article = treat_article(en_path,ENG_STOPWORDS,N_GRAM)
it_article = treat_article(it_path,ENG_STOPWORDS,N_GRAM)
es_article = treat_article(es_path,ENG_STOPWORDS,N_GRAM)
de_article = treat_article(de_path,ENG_STOPWORDS,N_GRAM)

france_lat_lon = (46.00,2.00)
spain_lat_lon = (40.46,-3.75)
italy_lat_lon = (42.23,13.57)
england_lat_lon = (52.35,-1.17)
de_lat_lon = (52.22, 4.53)

languages = ["french","english","italian","spanish", "deutch"]
articles = [fr_article,en_article,it_article,es_article, de_article]
lat_lon_values = [france_lat_lon,england_lat_lon,italy_lat_lon,spain_lat_lon, de_lat_lon]

100%|██████████| 324/324 [00:03<00:00, 81.20it/s] 


In [5]:
# Save the processed files 
for i,article in enumerate(articles):
    with open(f"./processed/{languages[i]}.pkl", mode = "wb") as f:
        pickle.dump((languages[i], article, lat_lon_values[i]),f)


In [2]:
# Load the processed files
languages = []
articles = []
lat_lon_values = []
for file in glob("./processed/*.pkl"):
    with open(file, mode = "rb") as f:
        lang, art, lat_lon = pickle.load(f)
    languages.append(lang)
    articles.append(art)
    lat_lon_values.append(lat_lon)

In [3]:
import helpers
viewer , score  = compute_plagiarism_from_articles(articles[2], articles[3], gap_tolerance=GAP_TOLERANCE, padding=PADDING)
viewer[15:18]

The two articles have a similarity score of 2.36, with 60 matching sequences. You can use the viewer to visualize the matching sequences

--- --- ---

The first article share 2.5336252736940885% of its content with the second
The second article share 2.1915584415584415% of its content with the first

--- --- ---



--- Printing matching n°15 ---
hereditary the fires children male legitimate of his subjects was afflicted to environment of them when they do able to succeed succeed to throne throne according to usage he was going to do their happiness Cardinal de Richeausii older than the monarchy The King of
ones sons male and legitimate they are crown like fiefdoms it was given with the obligation of the qualified to succeed succeed the throne throne according to a custom military service Geogr mod Tom IIyesCharlemagne i whichThe best cards that are been published The order


--- Printing matching n°16 ---
environment of them when they do able to succeed to throne according to usage he 

In [3]:
compute_many_to_many_sim(articles, languages, True, lat_lon_values)

        deutch   english    french   italian spanish
deutch     NaN       NaN       NaN       NaN     NaN
english    0.0       NaN       NaN       NaN     NaN
french     0.0  0.051176       NaN       NaN     NaN
italian    0.0  0.029895  2.362592       NaN     NaN
spanish    0.0  0.053762  3.985523  3.002979     NaN
